# NFL Data Pipeline, Visualization, and Machine Learning

Name: Aiden Hamade
Date: 03 December 2024

In this project, my goal is to take seasonal data from NFL teams and use it to predict whether or not a team will win against another team.
I am using seasonal data provided by *sumersports.com*, which includes the following stats for both the offense and defense:

- EPA/Play: Their expected points added per play
- Success %: Success rate is an advanced metric in football that measures efficiency, but with the important context of down and distance considered. A play is defined as successful if: It gains at least 50% of the yards required to move the chains on first down. 70% of yards to gain on second down
- EPA/Pass: Their expected points added per pass
- EPA/Rush: Their expected points added per rush
- Pass Yards: Total passing yards on the season
- Comp %: The total completions on the season
- Pass TD: Total passing touchdowns on the season
- Rush Yards: Total rushing yards on the season
- Rush TD’s: Total rush touchdowns on the season
- PROE: The pass rate minus their expected pass rate
- ADoT: The average depth of target
- Air EPA/Att: The EPA in the air from each pass
- Eckel %: The number of drives that end with a touchdown or have a first down snap within the 40 yard line divided by the number of total drives 
- Sack %: The amount of sacks divided by the amount of dropbacks
- Scramble %: The amount of scrambles divided by the amount of dropbacks
- INT %: Total interceptions / total pass attempts


In this project, I establish a data pipeline for NFL so that the data can easily be generated when needed. Then, I clean the data so that it is usable for visualization and machine learning tools.

I manipulate the data by adding in information from other sources. This data will allow me to use real NFL schedules as well as the winner of the game to be used for training/testing. I also manipulate the data by using the given data to generate better features to use in the machine learning model. For example, in order to reflect a good defense vs a good offense, I take metrics such as "Offensive EPA/Play" and add the "Defensive EPA/Play". In this example, if the offense of one team has an EPA/Play of 0.20, and then defense has an EPA/Play of -0.14, then the resulting EPA/Play of the offense is 0.06. I believe that this manipulation of the data will help to reflect that a good offense may play worse into a good defense.


# Reading in the Data

In this section of the code, I will read in offensive and defensive stats from seasons 2022-2024.
I store this in a pandas dataframe to be cleaned in the next code.

In [1]:
import pandas as pd
import numpy as np
import nest_asyncio
import asyncio
from playwright.async_api import async_playwright


In [2]:
"""
This section of data cleaning is necessary before downloading the data.
When I download the data, team names are prepended with their ranking, so when I download offense I may see
'1. Kansas City Chiefs' and defense may be '10. Kansas City Chiefs'.

By setting unique IDs, I am able to easily concatenate a team's offensive AND defensive stats for an entire season.
"""

# Creating unique IDs for each of the 32 teams

unique_ids = {
    'Cardinals': 0,
    'Falcons': 1,
    'Ravens': 2,
    'Bills': 3,
    'Panthers': 4,
    'Bears': 5,
    'Bengals': 6,
    'Browns': 7,
    'Cowboys': 8,
    'Broncos': 9,
    'Lions': 10,
    'Packers': 11,
    'Texans': 12,
    'Colts': 13,
    'Jaguars': 14,
    'Chiefs': 15,
    'Rams': 16,
    'Chargers': 17,
    'Raiders': 18,
    'Dolphins': 19,
    'Vikings': 20,
    'Patriots': 21,
    'Saints': 22,
    'Giants': 23,
    'Jets': 24,
    'Eagles': 25,
    'Steelers': 26,
    'Seahawks': 27,
    '49ers': 28,
    'Buccaneers': 29,
    'Titans': 30,
    'Commanders': 31
}

def teamUniqueId(df: pd.DataFrame) -> pd.DataFrame:
    # Convert team names to unique team IDs
    for idx, team in df['Team'].items():
        team_id = unique_ids[team.split()[-1]]
        df.loc[idx, 'Team'] = team_id
    
    return df

In [3]:
# Allow nested asyncio loops
nest_asyncio.apply()


async def fetch_table(url: str) -> pd.DataFrame:
    """
    This function allows me to scrape data from an online source.
    The table is generated via javascript, so I have to use the asyncio library in order to
    allow the page to load prior to scraping the table.

    Args:
        url (str): The url string where the data exists.

    Returns:
        table (pd.DataFrame): A pandas dataframe containing NFL data.
    """
    # Allow nested asyncio loops
    async with async_playwright() as p:
        # Launch browser
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        
        # Navigate to the URL
        url = url
        await page.goto(url)
        
        # Wait for the table to load (adjust selector as needed)
        await page.wait_for_selector('table')
        
        # Get the page content
        content = await page.content()
        await browser.close()
        
        # Parse the table with pandas
        tables = pd.read_html(content)
        return tables[0]  # Assuming the target table is the first one

def download_data(season: int) -> pd.DataFrame:
    """
    This function runs the fetch table function for both offensive and defensive data for an entire season.
    It then combines the offensive and defensive data into one dataframe.

    Args:
        season (int): The year of NFL seasonal data we want to scrape.
    
    Returns:
        season_table (pd.DataFrame): A pandas dataframe containing both offensive and defensive stats for all 32 teams for an entire season.
    """
    #Generate urls to extract offensive and defensive data
    offense_url = f'{base_url}/offensive/?season={season}'
    defense_url = f'{base_url}/defensive/?season={season}'
    
    # Fetch the offensive and defensive tables
    try:
        offense_table = asyncio.run(fetch_table(offense_url))
        defense_table = asyncio.run(fetch_table(defense_url))
    except Exception as e:
        print(f'Error: {e}')

    # Convert team names from strings to unique integers
    offense_table = teamUniqueId(offense_table)
    defense_table = teamUniqueId(defense_table)

    # Add prefixes to each column to differentiate between offensive and defensive data
    offense_table = offense_table.add_prefix("Offensive_")
    defense_table = defense_table.add_prefix("Defensive_")

    # Merge the offensive and defensive tables into one table
    season_table = pd.merge(offense_table, defense_table, left_on='Offensive_Team', right_on='Defensive_Team')

    # Drop redundant columns
    season_table.drop(columns=['Defensive_Team', 'Defensive_Season'])

    return season_table


base_url = f'https://sumersports.com/teams'
tables = []
start_year = 2022
end_year = 2024

# Iterate through each season we want to scrape data for and save the seasonal tables in a list
for i in range(start_year, end_year + 1):
    season_table = download_data(i)
    tables.append(season_table)

# Concatenate all of the seasons into one table
data = pd.concat(tables)

# Save the data to a csv file.
data.to_csv('seasonal_data.csv')
print('Data successfully downloaded.')

Data successfully downloaded.


# Cleaning the Data

In this section of the code I will clean the data. Some columns contain strings (i.e. Team, INT %, etc.) which need to be converted to floating point numbers.

Additionally, we will be removing rows with missing data.

In [4]:
def removePercentage(df: pd.DataFrame) -> pd.DataFrame:
    #Remove percent signs from columns and fix value
    df = df.apply(lambda col: col.replace('%', '', regex=True).astype(float))
    return df

In [5]:
from copy import deepcopy
cleaned_data = deepcopy(data)

cleaned_data = removePercentage(cleaned_data)

print(cleaned_data)

    Offensive_Team  Offensive_Season  Offensive_EPA/Play  Offensive_Success %  \
0             15.0            2022.0                0.17                 49.7   
1             25.0            2022.0                0.09                 47.3   
2              3.0            2022.0                0.09                 47.9   
3             28.0            2022.0                0.07                 45.5   
4              6.0            2022.0                0.06                 46.2   
..             ...               ...                 ...                  ...   
27             4.0            2024.0               -0.12                 40.2   
28             7.0            2024.0               -0.13                 37.4   
29            30.0            2024.0               -0.14                 38.4   
30             8.0            2024.0               -0.14                 39.5   
31            18.0            2024.0               -0.19                 39.5   

    Offensive_EPA/Pass  Off

# Normalize the Data

In this section of the code, I normalize all of the data to values between 0 and 1. I use a strategy called max-min scaling, where the maximum value in the column will result in 1, and the minimum value will result in 0.

This is a bit tricky using offensive and defensive data because offensive stats are better the more positive they are, and defensive stats are better the more negative they are. However, this is not true for all stats.

Better to be positive:
- Offensive EPA/Play
- Offensive Success %
- Offensive EPA/Pass
- Offensive EPA/Rush
- Offensive Pass Yards
- Offensive Comp %
- Offensive Pass TD
- Offensive Rush Yards
- Offensive Rush TD
- Offensive PROE
- Offensive ADoT
- Offensive YAC EPA/ATT
- Offensive Eckel %

Offense wants sack and int to be lower

Scramble isn't necessarily good or bad

Defense want sack and int to be higher, everything else lower

In [6]:
def normalize_data(df: pd.DataFrame, columns: list, reversed_columns: list) -> pd.DataFrame:
    """
    May want to adjust this function so that it is normalized to a specific season instead of all-time.
    """
    normalized_df = df.copy()
    
    # Normalize columns where higher is better
    for col in columns:
        normalized_df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())
    
    # Reverse normalize columns where lower is better
    for col in reversed_columns:
        #print(col)
        normalized_df[col] = 1 - (df[col] - df[col].min()) / (df[col].max() - df[col].min())
    
    return normalized_df

offensive_columns = ['Offensive_EPA/Play',
       'Offensive_Success %', 'Offensive_EPA/Pass', 'Offensive_EPA/Rush',
       'Offensive_Pass Yards', 'Offensive_Comp %', 'Offensive_Pass TD',
       'Offensive_Rush Yards', 'Offensive_Rush TD', 'Offensive_PROE',
       'Offensive_ADoT', 'Offensive_YAC EPA/Att', 'Offensive_Eckel %', 'Offensive_Scramble %']

offensive_reverse_columns = ['Offensive_Sack %', 'Offensive_INT %']

defensive_columns = ['Defensive_Sack %', 'Defensive_INT %']

defensive_reverse_columns = ['Defensive_EPA/Play',
       'Defensive_Success %', 'Defensive_EPA/Pass', 'Defensive_EPA/Rush',
       'Defensive_Pass Yards', 'Defensive_Comp %', 'Defensive_Pass TD',
       'Defensive_Rush Yards', 'Defensive_Rush TD', 'Defensive_PROE',
       'Defensive_ADoT', 'Defensive_YAC EPA/Att', 'Defensive_Eckel %', 'Defensive_Scramble %']


normal_data = normalize_data(cleaned_data, offensive_columns, offensive_reverse_columns)

normal_data = normalize_data(normal_data, defensive_columns, defensive_reverse_columns)

print(normal_data)

    Offensive_Team  Offensive_Season  Offensive_EPA/Play  Offensive_Success %  \
0             15.0            2022.0            0.914894             0.993377   
1             25.0            2022.0            0.744681             0.834437   
2              3.0            2022.0            0.744681             0.874172   
3             28.0            2022.0            0.702128             0.715232   
4              6.0            2022.0            0.680851             0.761589   
..             ...               ...                 ...                  ...   
27             4.0            2024.0            0.297872             0.364238   
28             7.0            2024.0            0.276596             0.178808   
29            30.0            2024.0            0.255319             0.245033   
30             8.0            2024.0            0.255319             0.317881   
31            18.0            2024.0            0.148936             0.317881   

    Offensive_EPA/Pass  Off

In [7]:
# Save the data to a CSV file
normal_data.to_csv('normal_data.csv')

# Machine Learning

Now that we have normalized all of the raw data to be used as features, we can test out some machine learning models.

In this section of the code, I will be using the sklearn library to predict the winner of games. In order to do so, I need to pull from a separate dataset that contains weekly game information. The winner of the weekly games will be used as the labels for the machine learning models.

We also need to further manipulate our data to reflect offensive/defensive matchups.
To do so, we will find the differential between an offense's seasonal stats and the defense's seasonal stats.
As mentioned at the beginning of the Jupyter Notebook, an example of this would be subtracting the Defensive EPA/Play from the Offensive EPA/Play, resulting in the Differential EPA/Play.

In [13]:
def fixIds(df: pd.DataFrame) -> pd.DataFrame:
    # Convert team names to unique team IDs
    for idx, team in df['team_home'].items():
        team_id = unique_ids[team.split()[-1]]
        df.loc[idx, 'team_home'] = team_id

    for idx, team in df['team_away'].items():
        team_id = unique_ids[team.split()[-1]]
        df.loc[idx, 'team_away'] = team_id
    
    return df

In [62]:
# Please update file_path to the correct file_path for your spreadspoke_scores.csv file.
# You can easily find this path by right clicking on the csv file and clicking 'Copy Path'
file_path = "/home/aiden/Senior Year Courses/DS501/spreadspoke_scores.csv"
data = pd.read_csv(file_path)

# Convert schedule_date to dtype datetime
data['schedule_date'] = pd.to_datetime(data['schedule_date'])

# Remove games that are missing feature data
feature_cols = ['schedule_playoff', 'schedule_season', 'spread_favorite', 'over_under_line', 'stadium_neutral', 'weather_wind_mph']
for col in feature_cols:
    data = data[data[col].notna()]


"""
In this section of the code, I am extracting data to be used as features in later code blocks.
I also derive labels from home/away scores and store those in a 1-column dataframe.
I use data from the 2020-2023 seasons because they don't have much missing data.
"""
#Grab data from the 2023 season
mask = (data['schedule_date'] >= '2022-09-08') & (data['schedule_date'] <= '2024-09-16')

#Slice the dataset
game_data = data[mask]
game_data.reset_index(inplace=True, drop=True)

# Create a dataframe for labels
labels = pd.DataFrame()

# Iterate through games and derive labels
# Home team win = 1, away team win/tie = 0
for idx, item in game_data['score_home'].items():
    home_score = item
    away_score = game_data.at[idx, 'score_away']
    if (home_score > away_score):
        labels.at[idx, 'Home Win'] = 1
    else:
        labels.at[idx, 'Home Win'] = 0

# Create a dataframe for features
feature_cols = ['team_home', 'team_away', 'schedule_playoff', 'schedule_season', 'spread_favorite', 'over_under_line', 'stadium_neutral', 'weather_wind_mph']

feature_data = data.loc[mask, feature_cols]
feature_data.reset_index(inplace=True, drop=True)

# Convert string True/False in schedule_playoff col into numeric values
for idx, item in feature_data['schedule_playoff'].items():
    if item == False:
        feature_data.at[idx, 'schedule_playoff'] = 0
    else:
        feature_data.at[idx, 'schedule_playoff'] = 1

# Convert string True/False in stadium_neutral col into numeric values
# if stadium is not neutral, set to 1 to signify home team advantage, else set to 0
for idx, item in feature_data['stadium_neutral'].items():
    if item == False:
        feature_data.at[idx, 'stadium_neutral'] = 1
    else:
        feature_data.at[idx, 'stadium_neutral'] = 0

feature_data = fixIds(feature_data)

feature_data = feature_data.astype(float)
print(feature_data)

     team_home  team_away  schedule_playoff  schedule_season  spread_favorite  \
0         16.0        3.0               0.0           2022.0             -2.0   
1          0.0       15.0               0.0           2022.0             -6.0   
2          1.0       22.0               0.0           2022.0             -5.5   
3          8.0       29.0               0.0           2022.0             -2.5   
4         10.0       25.0               0.0           2022.0             -5.5   
..         ...        ...               ...              ...              ...   
217        0.0       16.0               0.0           2024.0             -1.0   
218        8.0       22.0               0.0           2024.0             -6.5   
219       10.0       29.0               0.0           2024.0             -7.5   
220       12.0        5.0               0.0           2024.0             -6.0   
221       20.0       28.0               0.0           2024.0             -4.5   

     over_under_line  stadi

Iterate through every game.

Find the offensive stats and the defensive stats

Home_EPA/Play = home team's offensive EPA/Play - away team's defensive EPA/Play
Away_EPA/Play = away team's offensive EPA/Play - home team's defensive EPA/Play

In [90]:
# Iterate through every game
for idx, season in feature_data['schedule_season'].items():
    # home = home team, away = away team
    home = feature_data.loc[idx, 'team_home']
    away = feature_data.loc[idx, 'team_away']

    #Find offensive stats for home team
    home_stats = normal_data[(normal_data['Offensive_Team'] == home) & (normal_data['Offensive_Season'] == season)]
    home_stats.reset_index(inplace=True, drop=True)
    
    away_stats = normal_data[(normal_data['Offensive_Team'] == away) & (normal_data['Offensive_Season'] == season)]
    away_stats.reset_index(inplace=True, drop=True)


    home_offense_EPA = home_stats.at[0, 'Offensive_EPA/Play']
    away_defense_EPA = away_stats.at[0, 'Defensive_EPA/Play']
    home_EPA = home_offense_EPA - away_defense_EPA


    away_offense_EPA = away_stats.at[0, 'Offensive_EPA/Play']
    home_defense_EPA = home_stats.at[0, 'Defensive_EPA/Play']

    away_EPA = away_offense_EPA - home_defense_EPA

    feature_data.loc[idx, 'home_EPA'] = home_EPA
    feature_data.loc[idx, 'away_EPA'] = away_EPA

    print(f'home_offensive_EPA: {home_offense_EPA}\naway_offense_EPA: {away_offense_EPA}')
    print(f'home_defensive_EPA: {home_defense_EPA}\naway_defense_EPA: {away_defense_EPA}')
    print(f'home_EPA: {home_EPA}\naway_EPA: {away_EPA}')

    

home_offensive_EPA: 0.3617021276595745
away_offense_EPA: 0.7446808510638298
home_defensive_EPA: 0.4444444444444444
away_defense_EPA: 0.611111111111111
home_EPA: -0.24940898345153656
away_EPA: 0.30023640661938533
home_offensive_EPA: 0.3617021276595745
away_offense_EPA: 0.9148936170212768
home_defensive_EPA: 0.4722222222222222
away_defense_EPA: 0.5277777777777777
home_EPA: -0.1660756501182032
away_EPA: 0.4426713947990546
home_offensive_EPA: 0.5531914893617021
away_offense_EPA: 0.44680851063829796
home_defensive_EPA: 0.3055555555555556
away_defense_EPA: 0.6388888888888888
home_EPA: -0.08569739952718669
away_EPA: 0.14125295508274238
home_offensive_EPA: 0.574468085106383
away_offense_EPA: 0.46808510638297873
home_defensive_EPA: 0.75
away_defense_EPA: 0.5555555555555556
home_EPA: 0.01891252955082745
away_EPA: -0.28191489361702127
home_offensive_EPA: 0.6595744680851064
away_offense_EPA: 0.7446808510638298
home_defensive_EPA: 0.2777777777777777
away_defense_EPA: 0.6666666666666667
home_EPA: -0

In [91]:
print(feature_data)

     team_home  team_away  schedule_playoff  schedule_season  spread_favorite  \
0         16.0        3.0               0.0           2022.0             -2.0   
1          0.0       15.0               0.0           2022.0             -6.0   
2          1.0       22.0               0.0           2022.0             -5.5   
3          8.0       29.0               0.0           2022.0             -2.5   
4         10.0       25.0               0.0           2022.0             -5.5   
..         ...        ...               ...              ...              ...   
217        0.0       16.0               0.0           2024.0             -1.0   
218        8.0       22.0               0.0           2024.0             -6.5   
219       10.0       29.0               0.0           2024.0             -7.5   
220       12.0        5.0               0.0           2024.0             -6.0   
221       20.0       28.0               0.0           2024.0             -4.5   

     over_under_line  stadi

In [30]:
feature_data = feature_data.astype(float)
normal_data = normal_data.astype(float)

home_team_stats = pd.merge(feature_data, normal_data, left_on=['schedule_season', 'team_home'], right_on=['Offensive_Season', 'Offensive_Team'])

print(home_team_stats)

home_team_stats.to_csv('home_team.csv')

     team_home  team_away  schedule_playoff  schedule_season  spread_favorite  \
0         16.0        3.0               0.0           2022.0             -2.0   
1         16.0        1.0               0.0           2022.0            -10.0   
2         16.0        8.0               0.0           2022.0             -5.0   
3         16.0        4.0               0.0           2022.0            -10.0   
4         16.0       28.0               0.0           2022.0             -1.0   
..         ...        ...               ...              ...              ...   
217       22.0        4.0               0.0           2024.0             -4.0   
218        0.0       16.0               0.0           2024.0             -1.0   
219        8.0       22.0               0.0           2024.0             -6.5   
220       12.0        5.0               0.0           2024.0             -6.0   
221       20.0       28.0               0.0           2024.0             -4.5   

     over_under_line  stadi

In [95]:
df = pd.concat([feature_data, labels], axis=1)
print(df)

     team_home  team_away  schedule_playoff  schedule_season  spread_favorite  \
0         16.0        3.0               0.0           2022.0             -2.0   
1          0.0       15.0               0.0           2022.0             -6.0   
2          1.0       22.0               0.0           2022.0             -5.5   
3          8.0       29.0               0.0           2022.0             -2.5   
4         10.0       25.0               0.0           2022.0             -5.5   
..         ...        ...               ...              ...              ...   
217        0.0       16.0               0.0           2024.0             -1.0   
218        8.0       22.0               0.0           2024.0             -6.5   
219       10.0       29.0               0.0           2024.0             -7.5   
220       12.0        5.0               0.0           2024.0             -6.0   
221       20.0       28.0               0.0           2024.0             -4.5   

     over_under_line  stadi

In [100]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

#extract X (feature data) and y (data with labels)
X = feature_data.to_numpy()
y = labels.to_numpy().ravel()
print(f'X shape: {X.shape}, y shape: {y.shape}')

# Create a Random Forest model with 100 estimators
RF = RandomForestClassifier(100)

# Calculate scores for each fold using cross_val_score
scores = cross_val_score(RF, X, y, cv=5)

# Print scores
print(scores)

# Print mean accuracy score and standard deviation of each fold
print(f'Mean accuracy score: {scores.mean()}, Standard deviation: {scores.std()}')


X shape: (222, 10), y shape: (222,)
[0.62222222 0.66666667 0.68181818 0.68181818 0.70454545]
Mean accuracy score: 0.6714141414141414, Standard deviation: 0.027410054554944434
